In [1]:
#Computação científica
import numpy as np

#análise de dados
import pandas as pd

#visualização
import matplotlib.pyplot as plt

#machine learning
import sklearn

# feature engineering
from sklearn.impute import SimpleImputer
from feature_engine.imputation import (
    AddMissingIndicator)
from feature_engine.transformation import YeoJohnsonTransformer
from feature_engine.encoding import  RareLabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from feature_engine.wrappers import SklearnTransformerWrapper


#Pipelines
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

import gc

# Introdução

## Motivação


## Objetivos


# Parâmetros

Definição de alguns parâmetros que serão usados ao longo de todo o notebook

In [2]:
#Seed para o random state. Com o uso da mesma seed os resultados obtidos são reprodutíveis
seed=0

In [3]:
sklearn.set_config(display='diagram') #sklearn irá plotar diagramas
sklearn.set_config(transform_output="pandas") #output de aklearn são pandas data frames

# Download Dataset

In [4]:
# O dataset está disponível no seguinte URL:
#https://www.kaggle.com/competitions/porto-seguro-safe-driver-prediction/data

path='/home/rodolfo/Insync/rodolfopcruz2@gmail.com/Google Drive/Estudo/Python_Projects/datasets/porto-seguro-safe-driver-prediction/'
train=pd.read_csv(path+'x_train.csv')
val  = pd.read_csv(path+"x_val.csv")
test = pd.read_csv(path+'test.csv')

In [5]:
cat_features=[feature for feature in train.columns if 'cat' in feature]

bin_features=[feature for feature in train.columns if 'bin' in feature]

num_features=[feature for feature in train.columns if 'cat' not in feature 
                                                    and 'bin' not in feature and
                                                    feature!='target' and feature!='id']


In [6]:
y_train=train['target']
y_val  =val['target']
train  =train.drop(columns=['target','id'])
val    =val.drop(columns=['target','id'])
test   =test.drop(columns=["id"])

y_train.to_csv(path+'y_train.csv', index=False)  #salvar y_train separado do dataframe com os inputs
y_val.to_csv(path+'y_val.csv', index=False)

In [7]:
#Nos dados os missing values estão representados por -1
#Substituir -1 por np.nan para facilitar a identificação
train=train.replace(-1,np.nan)

# Pipelines

- Serão criadas duas pipelines de dados:

    - Em uma delas serão tratadas todas as colunas, a seleção das features será feita mais a frente;
    - Na outra a primeira etapa consistirá na remoção de algumas features, de acordo com os resultados obtidos com a correlação.

## Pipeline para dados com todas as features

In [8]:
#Nos dados os missing values foram substituídos pelo valor -1
#Função para trocar -1 por np .nan

def impute_nan_missing_value(x):
    x=x.replace(-1,np.nan)
    return x

impute_nan_missing_value_transformer=FunctionTransformer(impute_nan_missing_value)

In [9]:
#Função para converter o datatype para object

def converter_object(x):
    x=x.astype('object')
    return x

converter_object_transformer=FunctionTransformer(converter_object)

### Features Binárias

Pipeline para features binárias:

1) Substituir -1 por np.nan;
2) Converter os valores para object;
3) Substituir valores ausentes pelo mais frequente.

In [10]:
#imputer para substituir missing values
imputer_binary=SimpleImputer(strategy='most_frequent')

In [11]:
#Pipeline con três etapas
pipeline_bin_features=Pipeline([('replace_-1_nan',impute_nan_missing_value_transformer),
                          ('convert_to_object',converter_object_transformer),
                          ('fill_missing_binary',imputer_binary)])
pipeline_bin_features

Pipeline(steps=[('replace_-1_nan',
                 FunctionTransformer(func=<function impute_nan_missing_value at 0x7f453871e700>)),
                ('convert_to_object',
                 FunctionTransformer(func=<function converter_object at 0x7f456817d700>)),
                ('fill_missing_binary',
                 SimpleImputer(strategy='most_frequent'))])

### Features Numéricas

Pipeline para features numéricas:

1) Os missing values estão representados por -1. Substituir por np.nan;
2) Para as colunas numéricas com muitos missing values será criada um nova coluna para identificar um valor ausente;
3) Os valores ausentes de todas as colunas numéricas serão substituídos pela média;
4) Corrigir a distribuição de algumas colunas;
5) Normalização dos dados das colunas.


In [12]:
missing_threshold=1/100

#numeric features com muitos missing values
#coluna com número de missing values superior ao thresold estabelecido
num_features_muitos_na=train[num_features].isna().mean()>missing_threshold
num_features_muitos_na=num_features_muitos_na[num_features_muitos_na].index.to_list()

#numeric features com poucos missing values
num_features_poucos_na=train[num_features].isna().mean()<missing_threshold
num_features_poucos_na=num_features_poucos_na[num_features_poucos_na].index.to_list()

In [13]:
#Criar nova coluna para indicar se existe valor ausente nas colunas numéricas com muitos na

missing_indicator=AddMissingIndicator(variables=num_features_muitos_na)
#A coluna criada tem o mesmo nome da original acrescido da terminação _na
#A coluna criada é formada por 0 indicando ausência e 1 indicando presença de missing value

In [14]:
#Substituir os valores ausentes pela média
#Será aplicado a todas as colunas numéricas

num_imputer=SimpleImputer(strategy='mean')


In [15]:
#Aplicar transformação de Yeo Jhonson as seguintes features

features_to_be_transformed=['ps_reg_03',
                            'ps_car_12',
                            'ps_car_13',
                            'ps_car_14',
                            'ps_car_15',
                            'ps_reg_02']

yeo_transformer=YeoJohnsonTransformer(variables=features_to_be_transformed)

In [16]:
# Scaling

scaler = StandardScaler()

In [17]:
#Pipeline com 5 etapas

pipeline_num_features=Pipeline([
            ('replace_-1_nan',impute_nan_missing_value_transformer),
            ('addin_missing_indicator',missing_indicator),
            ('imputer_mean',SklearnTransformerWrapper(transformer=num_imputer,variables=num_features)),
            ('ajustar_ditribuicao',yeo_transformer),
            ('standard_scaler',SklearnTransformerWrapper(transformer=scaler,variables=num_features))])


### Features Categóricas

In [18]:
cat_features

['ps_ind_02_cat',
 'ps_ind_04_cat',
 'ps_ind_05_cat',
 'ps_car_01_cat',
 'ps_car_02_cat',
 'ps_car_03_cat',
 'ps_car_04_cat',
 'ps_car_05_cat',
 'ps_car_06_cat',
 'ps_car_07_cat',
 'ps_car_08_cat',
 'ps_car_09_cat',
 'ps_car_10_cat',
 'ps_car_11_cat']

In [19]:
missing_threshold=1/100
#features categóricas com número de missing values superior ao thresold
cat_features_muitos_na=train[cat_features].isna().mean()>missing_threshold
cat_features_muitos_na=cat_features_muitos_na[cat_features_muitos_na].index.to_list()

#features categóricas com número de missing values inferior ao thresold
cat_features_poucos_na=train[cat_features].isna().mean()<missing_threshold
cat_features_poucos_na=cat_features_poucos_na[cat_features_poucos_na].index.to_list()

In [20]:
#Substituir missing values nas features com muitos valores ausentes
cat_imputer_muitos_na=SimpleImputer(strategy='constant',fill_value='missing')

#Substituir missing values nas features com poucos valores ausentes
cat_imputer_poucos_na=SimpleImputer(strategy='most_frequent')


In [21]:
#Labels raras

rare_threshold=1/100
#Todas as labels que aparecem em proproção inferior a rare_thrshold serão agrupadas em uma única

#Identificar fearues que contem labels raras
rare_labels=[]
for feature in cat_features:
    if  not ((train[feature].value_counts()/len(train))>rare_threshold).all():
        rare_labels.append(feature)

rare_encoder = RareLabelEncoder(tol=rare_threshold, n_categories=1, variables=rare_labels)


In [22]:
#One hot encoder
enc = OneHotEncoder(handle_unknown='ignore',sparse_output=False)

#one hot encoding será aplicado somente a features com mais de duas labels
#e tambem aquelas com duas labels mas com muitos missing values, para a quais foi criada um label para indicar um missing value
columns_to_encode=[x for x in cat_features if train[x].nunique()>2 or x in cat_features_muitos_na]


In [23]:
#Converter para string para que tods os dados em uma mesma coluna tenham o mesmo formato
#Essa etapa é necessária para usar o one hot encoding

def converter_para_str(x):
    x=x.astype(str)
    return x

converter_str_transformer=FunctionTransformer(converter_para_str)

In [24]:
#Pipeline com 7 etapas
pipeline_cat_features=Pipeline([
            ('replace_-1_nan',impute_nan_missing_value_transformer),
            ('converter_object',converter_object_transformer),
            ('imputer_muitos_na',SklearnTransformerWrapper(transformer=cat_imputer_muitos_na,variables=cat_features_muitos_na)),
            ('imputer_poucos_na',SklearnTransformerWrapper(transformer=cat_imputer_poucos_na,variables=cat_features_poucos_na)),
            ('rare_labels',rare_encoder),
            ('converter_str',SklearnTransformerWrapper(transformer=converter_str_transformer,variables=columns_to_encode)),
            ('one_hot',SklearnTransformerWrapper(transformer=enc,variables=columns_to_encode))])

### Combinando as pipelines 

In [25]:
pipeline_bin_features=Pipeline([('replace_-1_nan',impute_nan_missing_value_transformer),
                                ('convert_to_object',converter_object_transformer),
                                ('fill_missing_binary',imputer_binary)])

pipeline_num_features=Pipeline([
            ('replace_-1_nan',impute_nan_missing_value_transformer),
            ('addin_missing_indicator',missing_indicator),
            ('imputer_mean',SklearnTransformerWrapper(transformer=num_imputer,variables=num_features)),
            ('ajustar_ditribuicao',yeo_transformer),
            ('standard_scaler',SklearnTransformerWrapper(transformer=scaler,variables=num_features))])

pipeline_cat_features=Pipeline([
            ('replace_-1_nan',impute_nan_missing_value_transformer),
            ('converter_object',converter_object_transformer),
            ('imputer_muitos_na',SklearnTransformerWrapper(transformer=cat_imputer_muitos_na,variables=cat_features_muitos_na)),
            ('imputer_poucos_na',SklearnTransformerWrapper(transformer=cat_imputer_poucos_na,variables=cat_features_poucos_na)),
            ('rare_labels',rare_encoder),
            ('converter_str',SklearnTransformerWrapper(transformer=converter_str_transformer,variables=columns_to_encode)),
            ('one_hot',SklearnTransformerWrapper(transformer=enc,variables=columns_to_encode))])

In [26]:
pipeline_dados_completos=ColumnTransformer(
    [('binary_features',pipeline_bin_features,bin_features),
     ('categorical_features',pipeline_cat_features,cat_features),
     ('numerical_features',pipeline_num_features,num_features)]
)

#### Transformação dos dados com a pipeline

In [47]:
#Carregar os dados novamente para que a pipeline seja aplicada aos dados sem nenhuma transformação 
path='/home/rodolfo/Insync/rodolfopcruz2@gmail.com/Google Drive/Estudo/Python_Projects/datasets/porto-seguro-safe-driver-prediction/'
train=pd.read_csv(path+'x_train.csv')
val  =pd.read_csv(path+'x_val.csv')
test = pd.read_csv(path+'test.csv')

In [28]:
#separar input do output
#remover coluna id
y_train=train['target']
y_val  =val  ['target']
train  =train.drop(columns=['target','id'])
val    =val.drop(columns=['target','id'])
test   =test.drop(columns=["id"])

In [29]:
pipeline_dados_completos.fit(train)

ColumnTransformer(transformers=[('binary_features',
                                 Pipeline(steps=[('replace_-1_nan',
                                                  FunctionTransformer(func=<function impute_nan_missing_value at 0x7f453871e700>)),
                                                 ('convert_to_object',
                                                  FunctionTransformer(func=<function converter_object at 0x7f456817d700>)),
                                                 ('fill_missing_binary',
                                                  SimpleImputer(strategy='most_frequent'))]),
                                 ['ps_ind_06_bin', 'ps_ind_07_bi...
                                                                                       'ps_calc_13',
                                                                                       'ps_calc_14']))]),
                                 ['ps_ind_01', 'ps_ind_03', 'ps_ind_14',
                                  'ps_ind_15', 'ps_reg_01', 'ps_reg_02',
                                  'ps_reg_03', 'ps_car_11', 'ps_car_12',
                                  'ps_car_13', 'ps_car_14', 'ps_car_15',
                                  'ps_calc_01', 'ps_calc_02', 'ps_calc_03',
                                  'ps_calc_04', 'ps_calc_05', 'ps_calc_06',
                                  'ps_calc_07', 'ps_calc_08', 'ps_calc_09',
                                  'ps_calc_10', 'ps_calc_11', 'ps_calc_12',
                                  'ps_calc_13', 'ps_calc_14'])])

In [30]:
#transformar dados de treino
train_complete_transformed=pipeline_dados_completos.transform(train)

In [31]:
#Salvar dados de treino transformados
train_complete_transformed.to_csv(path+'train_complete_transformed.csv', index=False)  

del train_complete_transformed
gc.collect()

58

In [32]:
#transformar dados de validação
val_complete_transformed=pipeline_dados_completos.transform(val)

In [33]:
#Salvar dados de validação transformados
val_complete_transformed.to_csv(path+'val_complete_transformed.csv', index=False)  

del val_complete_transformed
gc.collect()

41

In [34]:
#transformar dados de teste
test_complete_transformed=pipeline_dados_completos.transform(test)

In [35]:
#Salvar dados de teste transformados
test_complete_transformed.to_csv(path+'test_complete_transformed.csv', index=False)  

del test_complete_transformed
gc.collect()

41

## Pipeline para dados com features selecionadas

De acordo com os resultados verificados com a regressões realizadas na etapa de análise exploratória, serão removidas as features que não tem associação com o output e as features que tenham correlação elevada com outra.

- Features numéricas discretas sem correlação com o output:
    - ps_calc_02
    - ps_calc_03
    - ps_calc_04
    - ps_calc_05
    - ps_calc_06
    - ps_calc_07
    - ps_calc_08
    - ps_calc_09
    - ps_calc_10
    - ps_calc_11
    - ps_calc_12
    - ps_calc_13
    - ps_calc_14

- Features categóricas não associadas com output:
    - ps_car_05_cat
    - ps_car_10_cat

- Variáveis binárias não associadas com output:
    - ps_ind_10_bin
    - ps_ind_11_bin
    - ps_ind_13_bin
    - ps_calc_15_bin
    - ps_calc_16_bin
    - ps_calc_17_bin
    - ps_calc_18_bin
    - ps_calc_19_bin
    - ps_calc_19_bin
    - ps_calc_20_bin



In [49]:
#features numéricas que serão removidas
remove_num_features=['ps_calc_02',
    'ps_calc_03',
    'ps_calc_04',
    'ps_calc_05',
    'ps_calc_06',
    'ps_calc_07',
    'ps_calc_08',
    'ps_calc_09',
    'ps_calc_10',
    'ps_calc_11',
    'ps_calc_12',
    'ps_calc_13',
    'ps_calc_14']

#features categóricas que serão removidas

remove_cat_features=['ps_car_05_cat','ps_car_10_cat']


#features binárias que serão removidas
remove_bin_features=[
    'ps_ind_10_bin',
    'ps_ind_11_bin',
    'ps_ind_13_bin',
    'ps_calc_15_bin',
    'ps_calc_16_bin',
    'ps_calc_17_bin',
    'ps_calc_18_bin',
    'ps_calc_19_bin',
    'ps_calc_19_bin',
    'ps_calc_20_bin'
]



In [55]:
train=train.replace(-1,np.nan)

### Features Binárias

In [56]:
#features binárias que serão mantidas
remaining_bin_features=[feature for feature in bin_features if feature not in remove_bin_features]

In [57]:
#Não é necessário nenhuma alteração com relação a pipeline definida para os dados com todas as features
pipeline_bin_features=Pipeline([('replace_-1_nan',impute_nan_missing_value_transformer),
                          ('convert_to_object',converter_object_transformer),
                          ('fill_missing_binary',imputer_binary)])

In [58]:
pipeline_bin_features

Pipeline(steps=[('replace_-1_nan',
                 FunctionTransformer(func=<function impute_nan_missing_value at 0x7f453871e700>)),
                ('convert_to_object',
                 FunctionTransformer(func=<function converter_object at 0x7f456817d700>)),
                ('fill_missing_binary',
                 SimpleImputer(strategy='most_frequent'))])

### Features Numéricas

Será necessário alterar algumas das transformações aplicadas nos dados com todas as features, porque algumas das features as quais seriam aplicadas as transformações foram removidas

In [59]:
#Verificar os missing values nas features numéricas removidas

train[remove_num_features].isna().sum()/len(train)

ps_calc_02    0.0
ps_calc_03    0.0
ps_calc_04    0.0
ps_calc_05    0.0
ps_calc_06    0.0
ps_calc_07    0.0
ps_calc_08    0.0
ps_calc_09    0.0
ps_calc_10    0.0
ps_calc_11    0.0
ps_calc_12    0.0
ps_calc_13    0.0
ps_calc_14    0.0
dtype: float64

In [60]:
#Missing values em todas as features numéricas
train[num_features].isna().sum()/len(train)

ps_ind_01     0.000000
ps_ind_03     0.000000
ps_ind_14     0.000000
ps_ind_15     0.000000
ps_reg_01     0.000000
ps_reg_02     0.000000
ps_reg_03     0.181175
ps_car_11     0.000011
ps_car_12     0.000002
ps_car_13     0.000000
ps_car_14     0.071823
ps_car_15     0.000000
ps_calc_01    0.000000
ps_calc_02    0.000000
ps_calc_03    0.000000
ps_calc_04    0.000000
ps_calc_05    0.000000
ps_calc_06    0.000000
ps_calc_07    0.000000
ps_calc_08    0.000000
ps_calc_09    0.000000
ps_calc_10    0.000000
ps_calc_11    0.000000
ps_calc_12    0.000000
ps_calc_13    0.000000
ps_calc_14    0.000000
dtype: float64

In [61]:
#não foi removida nenhuma feature numérica com muitos valores ausentes 
#features numéricas que serão mantidas
remaining_num_features=[feature for feature in num_features if feature not in remove_num_features]

print('Número inicial de features numéricas: {}'.format(len(num_features)))
print('Número final de features numéricas após a remoção: {}'.format(len(remaining_num_features)))

Número inicial de features numéricas: 26
Número final de features numéricas após a remoção: 13


In [62]:
#numeric features com muitos missing values
num_features_muitos_na_remaining=train[remaining_num_features].isna().mean()>missing_threshold
num_features_muitos_na_remaining=num_features_muitos_na_remaining[num_features_muitos_na_remaining].index.to_list()

#numeric features com poucos missing values
num_features_poucos_na_remaining=train[remaining_num_features].isna().mean()<missing_threshold
num_features_poucos_na_remaining=num_features_poucos_na_remaining[num_features_poucos_na_remaining].index.to_list()

In [63]:
#Não foi removida nenhuma feature numérica com muitos na
print('Número inicial de features numéricas com muitos na: {}'.format(len(num_features_muitos_na)))
print('Número final de features numéricas com muitos na: {}'.format(len(num_features_muitos_na_remaining)))


Número inicial de features numéricas com muitos na: 2
Número final de features numéricas com muitos na: 2


In [64]:
#adicionar colunas indicando valor ausente  
missing_indicator=AddMissingIndicator(variables=num_features_muitos_na_remaining)
#substituir valor ausente pela média
num_imputer=SimpleImputer(strategy='mean')


In [65]:
#Transformação de Yeo Jhonson
#As features que serão tranformadas permanecem as mesmas
features_to_be_transformed=['ps_reg_03',
                            'ps_car_12',
                            'ps_car_13',
                            'ps_car_14',
                            'ps_car_15',
                            'ps_reg_02']

features_to_be_transformed_remaining=[feature for feature in features_to_be_transformed if feature in remaining_num_features] 
print('Número inicial de features que receberão a transformação de Yeo Jhonson: {}'.format(len(features_to_be_transformed)))
print('Número final de features que receberão a transformação de Yeo Jhonson: {}'.format(len(features_to_be_transformed)))

yeo_transformer=YeoJohnsonTransformer(variables=features_to_be_transformed_remaining)

Número inicial de features que receberão a transformação de Yeo Jhonson: 6
Número final de features que receberão a transformação de Yeo Jhonson: 6


In [66]:
# Scaling

scaler = StandardScaler()

In [67]:
#Não é necessário fazer nenhuma alteração na pipeline para dados numéricos definida anteriormente
pipeline_num_features=Pipeline([
            ('replace_-1_nan',impute_nan_missing_value_transformer),
            ('addin_missing_indicator',missing_indicator),
            ('imputer_mean',SklearnTransformerWrapper(transformer=num_imputer,variables=remaining_num_features)),
            ('ajustar_ditribuicao',yeo_transformer),
            ('standard_scaler',SklearnTransformerWrapper(transformer=scaler,variables=remaining_num_features))])
pipeline_num_features

Pipeline(steps=[('replace_-1_nan',
                 FunctionTransformer(func=<function impute_nan_missing_value at 0x7f453871e700>)),
                ('addin_missing_indicator',
                 AddMissingIndicator(variables=['ps_reg_03', 'ps_car_14'])),
                ('imputer_mean',
                 SklearnTransformerWrapper(transformer=SimpleImputer(),
                                           variables=['ps_ind_01', 'ps_ind_03',
                                                      'ps_ind_14', 'ps_ind_15',
                                                      'ps_reg_01', 'ps_reg_02',...
                 YeoJohnsonTransformer(variables=['ps_reg_03', 'ps_car_12',
                                                  'ps_car_13', 'ps_car_14',
                                                  'ps_car_15', 'ps_reg_02'])),
                ('standard_scaler',
                 SklearnTransformerWrapper(transformer=StandardScaler(),
                                           variables=['ps_ind_01', 'ps_ind_03',
                                                      'ps_ind_14', 'ps_ind_15',
                                                      'ps_reg_01', 'ps_reg_02',
                                                      'ps_reg_03', 'ps_car_11',
                                                      'ps_car_12', 'ps_car_13',
                                                      'ps_car_14', 'ps_car_15',
                                                      'ps_calc_01']))])

### Features Categóricas

In [68]:
#missing values na features categóricas
train[cat_features].isna().sum()/len(train)

ps_ind_02_cat    0.000368
ps_ind_04_cat    0.000145
ps_ind_05_cat    0.009828
ps_car_01_cat    0.000183
ps_car_02_cat    0.000008
ps_car_03_cat    0.690778
ps_car_04_cat    0.000000
ps_car_05_cat    0.447400
ps_car_06_cat    0.000000
ps_car_07_cat    0.019386
ps_car_08_cat    0.000000
ps_car_09_cat    0.000953
ps_car_10_cat    0.000000
ps_car_11_cat    0.000000
dtype: float64

In [69]:
#Missing values nas featues removidas
#ps_car_05_cat será mantida pq aparentemente a média do output é diferente comparando-se valores presentes e ausentes
train[remove_cat_features].isna().sum()/len(train)

ps_car_05_cat    0.4474
ps_car_10_cat    0.0000
dtype: float64

In [70]:
remove_cat_features=['ps_car_10_cat'] #somente ps_car_10_cat será removida
remaining_cat_features=[feature for feature in cat_features if feature not in remove_cat_features]

In [71]:
missing_threshold=1/100
#features categóricas com número de missing values superior ao thresold
cat_features_muitos_na_remaining=train[remaining_cat_features].isna().mean()>missing_threshold
cat_features_muitos_na_remaining=cat_features_muitos_na_remaining[cat_features_muitos_na_remaining].index.to_list()

#features categóricas com número de missing values inferior ao thresold
cat_features_poucos_na_remaining=train[remaining_cat_features].isna().mean()<missing_threshold
cat_features_poucos_na_remaining=cat_features_poucos_na_remaining[cat_features_poucos_na_remaining].index.to_list()

In [72]:
#Substituir missing values nas features com muitos valores ausentes
cat_imputer_muitos_na=SimpleImputer(strategy='constant',fill_value='missing')

#Substituir missing values nas features com poucos valores ausentes
cat_imputer_poucos_na=SimpleImputer(strategy='most_frequent')

In [73]:
#Labels raras

rare_threshold=1/100
#Todas as labels que aparecem em proproção inferior a rare_thrshold serão agrupadas como uma única

#Identificar fearues que contem labels raras
rare_labels_remaining=[]
for feature in remaining_cat_features:
    if  not ((train[feature].value_counts()/len(train))>rare_threshold).all():
        rare_labels_remaining.append(feature)

rare_encoder_remaining = RareLabelEncoder(tol=rare_threshold, n_categories=1, variables=rare_labels_remaining)


In [74]:
#One hot encoder
enc = OneHotEncoder(handle_unknown='ignore',sparse_output=False)

#one hot encoding será aplicado somente a features com mais de duas labels
#e tambem aquelas com duas labels mas com muitos missing values, para a quais foi criada um label para indicar um missing value
columns_to_encode_remaining=[x for x in remaining_cat_features if train[x].nunique()>2 or x in cat_features_muitos_na_remaining]


In [75]:
#Converter para string para que tods os dados em uma mesma coluna tenham o mesmo formato
#Essa etapa é necessária para usar o one hot encoding

def converter_para_str(x):
    x=x.astype(str)
    return x

converter_str_transformer=FunctionTransformer(converter_para_str)

In [76]:
pipeline_cat_features_remaining=Pipeline([
            ('replace_-1_nan',impute_nan_missing_value_transformer),
            ('converter_object',converter_object_transformer),
            ('imputer_muitos_na',SklearnTransformerWrapper(transformer=cat_imputer_muitos_na,variables=cat_features_muitos_na_remaining)),
            ('imputer_poucos_na',SklearnTransformerWrapper(transformer=cat_imputer_poucos_na,variables=cat_features_poucos_na_remaining)),
            ('rare_labels',rare_encoder_remaining),
            ('converter_str',SklearnTransformerWrapper(transformer=converter_str_transformer,variables=columns_to_encode_remaining)),
            ('one_hot',SklearnTransformerWrapper(transformer=enc,variables=columns_to_encode_remaining))])

### Combinando as pipelines 

In [77]:
pipeline_bin_features_remaining=Pipeline([('replace_-1_nan',impute_nan_missing_value_transformer),
                                ('convert_to_object',converter_object_transformer),
                                ('fill_missing_binary',imputer_binary)])

pipeline_num_features_remaining=Pipeline([
            ('replace_-1_nan',impute_nan_missing_value_transformer),
            ('addin_missing_indicator',missing_indicator),
            ('imputer_mean',SklearnTransformerWrapper(transformer=num_imputer,variables=remaining_num_features)),
            ('ajustar_ditribuicao',yeo_transformer),
            ('standard_scaler',SklearnTransformerWrapper(transformer=scaler,variables=remaining_num_features))])

pipeline_cat_features_remaining=Pipeline([
            ('replace_-1_nan',impute_nan_missing_value_transformer),
            ('converter_object',converter_object_transformer),
            ('imputer_muitos_na',SklearnTransformerWrapper(transformer=cat_imputer_muitos_na,variables=cat_features_muitos_na_remaining)),
            ('imputer_poucos_na',SklearnTransformerWrapper(transformer=cat_imputer_poucos_na,variables=cat_features_poucos_na_remaining)),
            ('rare_labels',rare_encoder_remaining),
            ('converter_str',SklearnTransformerWrapper(transformer=converter_str_transformer,variables=columns_to_encode_remaining)),
            ('one_hot',SklearnTransformerWrapper(transformer=enc,variables=columns_to_encode_remaining))])

In [78]:
#Criar um transformer para remover colunas desnecessárias

def remover_colunas(x,columns):
    
    '''
    Inputs:
        x       - dataframe original
       columns  - lista com colunas que serão mantidas
    '''

    x=x.loc[:,columns]
    return x


In [79]:
#TRansformação para remover features
pipeline_remover_features=Pipeline([('remover_features',FunctionTransformer(remover_colunas,kw_args={'columns':remaining_bin_features+
                                                                                                                remaining_cat_features+
                                                                                                                remaining_num_features}))])

In [80]:
#Transformações que serão aplicadas aos dados restantes
pipeline_transformacoes_features_selecionadas=ColumnTransformer([('features binarias',pipeline_bin_features_remaining,remaining_bin_features),
                                                     ('features numericas',pipeline_num_features_remaining,remaining_num_features),
                                                     ('features categoricas',pipeline_cat_features_remaining,remaining_cat_features)])

In [81]:
#Pipeline que combina a seleção das features com as transformações
pipeline_features_selecionadas=Pipeline([('selecionar_features',pipeline_remover_features),
                                         ('transformar_features',pipeline_transformacoes_features_selecionadas)])

In [82]:
pipeline_features_selecionadas

Pipeline(steps=[('selecionar_features',
                 Pipeline(steps=[('remover_features',
                                  FunctionTransformer(func=<function remover_colunas at 0x7f45279bca60>,
                                                      kw_args={'columns': ['ps_ind_06_bin',
                                                                           'ps_ind_07_bin',
                                                                           'ps_ind_08_bin',
                                                                           'ps_ind_09_bin',
                                                                           'ps_ind_12_bin',
                                                                           'ps_ind_16_bin',
                                                                           'ps_ind_17_bin',
                                                                           'ps_ind_18_bin',
                                                                           'ps_ind_02_cat',
                                                                           'ps_ind_04_cat',
                                                                           'ps_ind_05_cat',
                                                                           'ps_car_01_c...
                                                                                                        'ps_ind_05_cat',
                                                                                                        'ps_car_01_cat',
                                                                                                        'ps_car_03_cat',
                                                                                                        'ps_car_04_cat',
                                                                                                        'ps_car_05_cat',
                                                                                                        'ps_car_06_cat',
                                                                                                        'ps_car_07_cat',
                                                                                                        'ps_car_09_cat',
                                                                                                        'ps_car_11_cat']))]),
                                                  ['ps_ind_02_cat',
                                                   'ps_ind_04_cat',
                                                   'ps_ind_05_cat',
                                                   'ps_car_01_cat',
                                                   'ps_car_02_cat',
                                                   'ps_car_03_cat',
                                                   'ps_car_04_cat',
                                                   'ps_car_05_cat',
                                                   'ps_car_06_cat',
                                                   'ps_car_07_cat',
                                                   'ps_car_08_cat',
                                                   'ps_car_09_cat',
                                                   'ps_car_11_cat'])]))])

#### Transformação dos dados com a pipeline

In [83]:
#Carregar os dados novamente para que a pipeline seja aplicada aos dados sem nenhuma transformação 
path='/home/rodolfo/Insync/rodolfopcruz2@gmail.com/Google Drive/Estudo/Python_Projects/datasets/porto-seguro-safe-driver-prediction/'
train=pd.read_csv(path+'x_train.csv')
val  =pd.read_csv(path+'x_val.csv')
test = pd.read_csv(path+'test.csv')

In [84]:
#separar input do output
#remover coluna id

y_train=train['target']
y_val  =val  ['target']
train  =train.drop(columns=['target','id'])
val    =val.drop(columns=['target','id'])
test   =test.drop(columns=["id"])

In [85]:
pipeline_features_selecionadas.fit(train)

Pipeline(steps=[('selecionar_features',
                 Pipeline(steps=[('remover_features',
                                  FunctionTransformer(func=<function remover_colunas at 0x7f45279bca60>,
                                                      kw_args={'columns': ['ps_ind_06_bin',
                                                                           'ps_ind_07_bin',
                                                                           'ps_ind_08_bin',
                                                                           'ps_ind_09_bin',
                                                                           'ps_ind_12_bin',
                                                                           'ps_ind_16_bin',
                                                                           'ps_ind_17_bin',
                                                                           'ps_ind_18_bin',
                                                                           'ps_ind_02_cat',
                                                                           'ps_ind_04_cat',
                                                                           'ps_ind_05_cat',
                                                                           'ps_car_01_c...
                                                                                                        'ps_ind_05_cat',
                                                                                                        'ps_car_01_cat',
                                                                                                        'ps_car_03_cat',
                                                                                                        'ps_car_04_cat',
                                                                                                        'ps_car_05_cat',
                                                                                                        'ps_car_06_cat',
                                                                                                        'ps_car_07_cat',
                                                                                                        'ps_car_09_cat',
                                                                                                        'ps_car_11_cat']))]),
                                                  ['ps_ind_02_cat',
                                                   'ps_ind_04_cat',
                                                   'ps_ind_05_cat',
                                                   'ps_car_01_cat',
                                                   'ps_car_02_cat',
                                                   'ps_car_03_cat',
                                                   'ps_car_04_cat',
                                                   'ps_car_05_cat',
                                                   'ps_car_06_cat',
                                                   'ps_car_07_cat',
                                                   'ps_car_08_cat',
                                                   'ps_car_09_cat',
                                                   'ps_car_11_cat'])]))])

In [86]:
#transformar dados de treino
train_selected_features_transformed=pipeline_features_selecionadas.transform(train)

In [87]:
#salvar dados de treino
train_selected_features_transformed.to_csv(path+'train_selected_features_transformed.csv', index=False)  
del train_selected_features_transformed
gc.collect()

66

In [88]:
#transformar dados de validação
val_selected_features_transformed=pipeline_features_selecionadas.transform(val)

In [89]:
#salvar dados de validacao
val_selected_features_transformed.to_csv(path+'val_selected_features_transformed.csv', index=False)  
del val_selected_features_transformed
gc.collect()

41

In [90]:
#transformar dados de teste
test_selected_features_transformed=pipeline_features_selecionadas.transform(test)

In [81]:
#salvar dados de teste
test_selected_features_transformed.to_csv(path+'test_selected_features_transformed.csv', index=False)  
